In [ ]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
import optuna
import pickle
from datetime import timedelta
import time
import os 
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import gc
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
feat = pd.read_csv('./data/test.csv')
FEATURE_COLS = feat.columns[1:].tolist()

In [ ]:
study_name = '530_yritys'

In [ ]:


pickle_file_path = f'./data/train_df_neljas.pickle'

with open(pickle_file_path, 'rb') as f:
    train_df = pickle.load(f)

In [ ]:
# # model_features_514_convnextlarge_maxavg_2	model_features_511_convnextlarge_3	523_ConvNeXtXLarge_4	model_features_523_ConvNeXtXLarge_new	527_Convnextlarge	model_features_527_convnextl_0
train_df.drop(['523_ConvNeXtXLarge_4', '527_Convnextlarge', 'model_features_514_convnextlarge_maxavg_2'], axis=1, inplace=True)

In [ ]:
pd.set_option('display.max_rows', None)

# Aseta näyttämään rajoittamaton määrä sarakkeita
pd.set_option('display.max_columns', None)

In [ ]:
train_df.head()

In [ ]:
num_total = train_df['model_features_527_convnextl_0'].iloc[0]
num_total = len(num_total)
print(f'num_total {num_total}')

In [ ]:
num_total = train_df['model_features_514_convnextlarge_maxavg_2'].iloc[0]
num_total = len(num_total)
print(f'num_total {num_total}')

In [ ]:
num_total = train_df['model_features_511_convnextlarge_3'].iloc[0]
num_total = len(num_total)
print(f'num_total {num_total}')

In [ ]:
num_total = train_df['model_features_523_ConvNeXtXLarge_new'].iloc[0]
num_total = len(num_total)
print(f'num_total {num_total}')

In [ ]:
print(train_df['fold'].value_counts())

In [ ]:
# # model_features_514_convnextlarge_maxavg_2	model_features_511_convnextlarge_3	523_ConvNeXtXLarge_4	model_features_523_ConvNeXtXLarge_new	527_Convnextlarge	model_features_527_convnextl_0

# def prepare_features(df, choose_features):

#     # print(f'In prepare_features: {choose_features}')

#     if choose_features['features'] == "on":
#         data = [df[col].values for col in FEATURE_COLS]
#     else:
#         data = []
    
#     print(f'In prepare_features: {len(data)}')
#     print(f'data type in prepare_features: {type(data)}')

#     for key, value in choose_features.items():

#         if key == "features":
#             continue

#         if value == "on":
#             print(f'selected feature: {key}')
#             print(f'df key: {np.vstack(df[key].values).shape}')
#             data.append(np.vstack(df[key].values))
#             print(f'data len in prepare_features: {len(data)}')

    
#     all_features = np.column_stack(data).tolist()
#     count_on = sum(1 for value in choose_features.values() if value == 'on')
#     print(f'count_on: {count_on}')
    
#     df['all_features'] = all_features
    
#     return df


In [ ]:
# def prepare_features(df, choose_features):
#     data = []
    
#     # Vain jos 'features' on 'on', lisää FEATURE_COLS datalistalle
#     if choose_features['features'] == "on":
#         print(f'features on')
#         data.extend([df[col].values for col in FEATURE_COLS])
        

#     # Lisää muut 'on' tilassa olevat featuret
#     for key, value in choose_features.items():
#         if key != "features" and value == "on":
#             feature_data = df[key].values

#             print(f'with key {key} and value {value}')
#             print(f'feature_data shape: {feature_data.shape}')

#             if feature_data.ndim == 1:
#                 feature_data = feature_data[:, np.newaxis]  # Varmista, että data on 2D
#             data.append(feature_data)
    
    
#     all_features = np.column_stack(data).tolist()
    
    

#     df['all_features'] = all_features
#     df.head()
#     # return df


In [ ]:
# def prepare_features(df, choose_features):
#     data = []

#     if choose_features.get('features', 'off') == "on":
#         data.extend([df[col].values[:, np.newaxis] if df[col].values.ndim == 1 else df[col].values for col in FEATURE_COLS])

#     for key, value in choose_features.items():
#         if key != "features" and value == "on":
#             feature_data = df[key].values
#             if feature_data.ndim == 1:
#                 feature_data = feature_data[:, np.newaxis]  # Varmista, että data on 2D
#             data.append(feature_data)

#     if data:
#         all_features = np.hstack(data)
#     else:
#         all_features = np.empty((df.shape[0], 0))  # Tyhjä taulukko, jos ei featureita

#     df['all_features'] = list(all_features)
#     return df


In [ ]:
# import numpy as np

# def prepare_features(df, choose_features):
    

#     # Käsitellään FEATURE_COLS, jos 'features' avain on 'on'
#     if choose_features.get('features', 'off') == "on":
#          data = [df[col].values for col in FEATURE_COLS]
#     else:
#         data = []

#     # Lisätään valitut featuret perustuen choose_features ohjeisiin
#     for key, value in choose_features.items():
#         if key != "features" and value == "on" and key in df:
#             feature_data = df[key].values.reshape(-1, 1)
#             data.append(feature_data)

#     # Varmistetaan, että data on olemassa ennen yhdistämistä
#     if data:
#         all_features = np.hstack(data)
#     else:
#         # Luodaan tyhjä taulukko, jos ei featureita
#         all_features = np.empty((len(df), 0))

#     # Asetetaan yhdistetyt featuret DataFrameen
#     df['all_features'] = list(all_features)

#     return df


In [ ]:
def prepare_features(df, feature_columns, features):

    # print(f'In prepare_features columns: {feature_columns}')
    # print(f'In prepare_features features: {features}')
    # Yhdistää useita sarakkeita, oletetaan että jokainen arvo on listamuodossa tai pienenä NumPy-taulukkona
    if features == "on":
        data = [df[col].values for col in FEATURE_COLS]
    else:
        data = []
    
    if len(feature_columns) != 0:
        combined_features = np.hstack([np.vstack(df[col].values) for col in feature_columns])
        data.append(combined_features)
    
    all_features = np.column_stack(data).tolist()
    
    df['all_features'] = all_features
    return df


In [ ]:
import warnings

# Ohita tietyn tyyppiset varoitukset
warnings.filterwarnings('ignore', category=UserWarning)


def get_features_array(features_series):
    # Muuntaa sarjan, joka sisältää taulukoita, yhdeksi 2D-taulukoksi
    return np.array(list(features_series))


def objective(trial, df, target):
    param = {        
        'objective': 'reg:squarederror',
        'device' : 'cuda',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log = True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log = True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0, log = True),        
        'max_depth': trial.suggest_int('max_depth', 2, 3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 142),
        'feature_selector': trial.suggest_categorical('feature_selector', ['shuffle', 'greedy', 'thrifty', 'cyclic', 'random']),
        'boosting': trial.suggest_categorical('boosting', ['gbtree', 'gblinear', 'dart'])
        }
    

    # # model_features_514_convnextlarge_maxavg_2	model_features_511_convnextlarge_3	523_ConvNeXtXLarge_4	model_features_523_ConvNeXtXLarge_new	527_Convnextlarge	model_features_527_convnextl_0
    feature_names = ['model_features_511_convnextlarge_3', 'model_features_523_ConvNeXtXLarge_new', 'model_features_527_convnextl_0']
    choosed_features = []

    features = trial.suggest_categorical('features', ['on', 'off'])
    
    for col in feature_names:
        if trial.suggest_categorical(col, ['on', 'off']) == 'on':
            choosed_features.append(col)
    
    print(f'len choosed_features {len(choosed_features)}')

    if features == 'off':
        if len(choosed_features) == 0:
            print('No features selected')
            raise optuna.TrialPruned()

    df_this = prepare_features(df,choosed_features, features)
    
    
    num_total = df_this['all_features'].iloc[0]
    num_total = len(num_total)
    print(f'num_total {num_total}')
    
    folds = [0,2]
    mse_scores = []
    r2_scores = []

    for fold in folds:

    
        train_data = df_this[df_this['fold'] != fold]
        valid_data = df_this[df_this['fold'] == fold]

    
        X_train = get_features_array(train_data['all_features'])
        X_valid = get_features_array(valid_data['all_features'])
        print(f'Shape X_train {X_train.shape}')

        y_train = train_data[target]
        y_valid = valid_data[target]

        print(f'Y_train shape {y_train.shape}')

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_valid, label=y_valid)

        print(f'Done creating DMatrix')

        num_boost_round = trial.suggest_int('n_estimators', 10, 20, log=True) 
        
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        print(f'Starting training')
        model = xgb.train(param, dtrain, num_boost_round=num_boost_round, evals=watchlist, verbose_eval=False)
        preds = model.predict(dvalid)
        mse = mean_squared_error(y_valid, preds)
        r2 = r2_score(y_valid, preds)

        trial.report(r2, fold)

        if trial.should_prune():
            print(f'Pruned fold {fold} with value {r2} and mse {mse}')
            raise optuna.TrialPruned()

        print(f'Fold {fold} MSE: {mse} R2: {r2}')
        mse_scores.append(mse)
        r2_scores.append(r2)

        del model, dtrain, dvalid, X_train, X_valid, y_train, y_valid, train_data, valid_data
        
        tf.keras.backend.clear_session()
        gc.collect()
    
    
    del df_this
    tf.keras.backend.clear_session()
    gc.collect()
    
    return np.mean(r2_scores)
    

def optimize_model(df, target):

    if os.path.exists(f'./NN_search/{study_name}_{target}_qmc_sampler.pickle'):
        with open(f'./NN_search/{study_name}_{target}_qmc_sampler.pickle', 'rb') as f:
            print(f'Loading QMC sampler from file {f}')
            qmc_sampler = pickle.load(f)
    else:
        print(f'Creating new QMC sampler')
        qmc_sampler = optuna.samplers.QMCSampler(warn_independent_sampling = False)

    if os.path.exists(f'./NN_search/{study_name}_{target}_tpe_sampler.pickle'):
        with open(f'./NN_search/{study_name}_{target}_tpe_sampler.pickle', 'rb') as f:
            print(f'Loading TPE sampler from file {f}')
            tpe_sampler = pickle.load(f)
    else:
        print(f'Creating new TPE sampler')
        tpe_sampler = optuna.samplers.TPESampler(n_startup_trials=0, multivariate=True, warn_independent_sampling = False)

    if os.path.exists(f'./NN_search/{study_name}_{target}_pruner.pickle'):
        with open(f'./NN_search/{study_name}_{target}_pruner.pickle', 'rb') as f:
            print(f'Loading pruner from file {f}')
            pruner = pickle.load(f)
    else:
        print(f'Creating new pruner')
        pruner = optuna.pruners.MedianPruner(n_startup_trials=5)

    start_time = time.time()
    study = optuna.create_study(direction='maximize',
                            study_name=study_name,
                            storage=f'sqlite:///testi3_{target}.db',
                            load_if_exists=True                                    
                            )
    
    print(f'Starting optimization for {target} with qmc sampler')
    random_time = time.time()
    study.sampler = qmc_sampler
    study.optimize(lambda trial: objective(trial, df, target), n_trials=5)
    print(f'QCM optimization finished in {timedelta(seconds=time.time() - random_time)}')

    print(f'Saving QMC sampler to file ./NN_search/{study_name}_{target}_qmc_sampler.pickle')
    with open(f'./NN_search/{study_name}_{target}_qmc_sampler.pickle', 'wb') as f:
        pickle.dump(qmc_sampler, f)

    print(f'Starting optimization for {target} with TPE sampler')
    tpe_time = time.time()
    study.sampler = tpe_sampler
    study.optimize(lambda trial: objective(trial, df, target), n_trials=20)
    print(f'TPE optimization finished in {timedelta(seconds=time.time() - tpe_time)}')

    print(f'Saving TPE sampler to file ./NN_search/{study_name}_{target}_tpe_sampler.pickle')
    with open(f'./NN_search/{study_name}_{target}_tpe_sampler.pickle', 'wb') as f:
        pickle.dump(tpe_sampler, f)

    print(f'Saving pruner to file ./NN_search/{study_name}_{target}_pruner.pickle')
    with open(f'./NN_search/{study_name}_{target}_pruner.pickle', 'wb') as f:
        pickle.dump(pruner, f)

    print(f'Optimization finished in {timedelta(seconds=time.time() - start_time)}')

    
    



In [ ]:
train_df.head()

In [ ]:
target_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
train_fold = 80
validation_fold = 1

time_search_start = time.time()
time_taken = 0

while time_taken < 3600 * 3:
    for target in target_columns:    
        print(f'\n\nOptimizing model for {target}\n\n')
        optimize_model(train_df, target)
        time_taken = time.time() - time_search_start
        print(f'Time taken: {timedelta(seconds=time_taken)}')   

In [ ]:
def get_features_array(features_series):
    # Muuntaa sarjan, joka sisältää taulukoita, yhdeksi 2D-taulukoksi
    return np.array(list(features_series))


target_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']

best_r2 = []

train_pred = np.zeros((train_df.shape[0], len(target_columns)))

selectors = {}
models = {}

for i, target in enumerate(target_columns):    

    tf.keras.backend.clear_session()
    gc.collect()
    
    study = optuna.create_study(direction='maximize',
                        study_name=study_name,
                        storage=f'sqlite:///530_yritys_{target}.db',
                        load_if_exists=True                            
                            )
    
    best_params = study.best_trial.params

    print(f'Best value {study.best_value}')    
    print(f'Num trials {len(study.trials)}')

    best_r2.append(study.best_value)

    choosed_features = []

    features = study.best_params['features']

    feature_names = ['model_features_511_convnextlarge_3', 'model_features_523_ConvNeXtXLarge_new', 'model_features_527_convnextl_0']

    for col in feature_names:
        if study.best_params[col] == 'on':
            choosed_features.append(col)
    
    print(f'len choosed_features {len(choosed_features)}')

    df_features = prepare_features(train_df, choosed_features, features)
    X_selected = get_features_array(df_features['all_features'])

    print(f'X_selected shape {X_selected.shape}')


    best_params['objective'] = 'reg:squarederror'
    best_params['device'] = 'cuda'
    print(f'Best params for {target}: {best_params}')
    
    for param in ['features', 'model_features_511_convnextlarge_3', 'model_features_523_ConvNeXtXLarge_new', 'model_features_527_convnextl_0', 'n_estimators']:
        best_params.pop(param, None)

    best_params.pop('boosting')
    best_params.pop('feature_selector')
    best_params['booster'] = 'dart'


    model = xgb.XGBRegressor(**best_params, verbosity=1)  
    model.fit(X_selected, train_df[target])
    models[target] = model
    
    pred = models[target].predict(X_selected)
    r2 = r2_score(train_df[target], pred)
    mse = mean_squared_error(train_df[target], pred)
    print(f'Training R2 for {target}: {r2} and MSE: {mse}')

    print(f'Model for {target} {models[target]}')

    train_pred[:, i] = models[target].predict(X_selected)


train_r2 = r2_score(train_df[target_columns], train_pred)
print(f'Training R2: {train_r2}')

for idx, name in enumerate(target_columns):
    print(f'Target {name} R2: {best_r2[idx]}')

In [ ]:
del train_df
gc.collect()

pickle_file_path = f'./data/test_df_neljas.pickle'

with open(pickle_file_path, 'rb') as f:
    test_df = pickle.load(f)

In [ ]:
test_df.head()

In [ ]:


test_preds = np.zeros((len(test_df), len(target_columns)))

for i, target in enumerate(target_columns):

    print(f'Predicting {target} with model {models[target]}')

    study = optuna.create_study(direction='maximize',
                    study_name=study_name,
                    storage=f'sqlite:///530_yritys_{target}.db',
                    load_if_exists=True                            
                        )

    feature_names = ['model_features_511_convnextlarge_3', 'model_features_523_ConvNeXtXLarge_new', 'model_features_527_convnextl_0']

    for col in feature_names:
        if study.best_params[col] == 'on':
            choosed_features.append(col)
    
    print(f'len choosed_features {len(choosed_features)}')

    df_features = prepare_features(test_df, choosed_features, features)
    X_selected = get_features_array(df_features['all_features'])

    print(f'X_selected shape {X_selected.shape}')
    
    test_preds[:, i] = models[target].predict(X_selected)
     

In [ ]:
target_columns = ['X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

test_df_copy = test_df.copy()
submission_df = test_df_copy[['id']].copy()
submission_df[target_columns] = test_preds

In [ ]:
submission_df.describe()

In [ ]:
target_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
train_df[target_columns].describe()

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('./data/submission.csv', index=False)